In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import xgboost
import pandas as pd 
import PIL
! pip install shap
! pip install trulens
import shap
import trulens
import requests

# Local Surrogate Models (LIME)

In [ ]:
def f(x):
    if x < -0.1:
        return -10 + 1.2806 * x**2
    else:
        return x
f = np.vectorize(f)

def weighted_neighborhood_LIME(query, comparison_X, comparison_Y, d):
    # x is the query instance. because our function is univariate, it is a single number
    # d is the neighborhood size (int/float) 
    # comparison_X is a 1-D numpy array of points around query that are used in LIME
    # comparison_Y is a 1-D numpy array of the corresponding target variables for comparison_X
    # << YOUR CODE HERE >> 

In [ ]:
X = np.arange(-10, 10, 0.1)
Y = f(X)
print('d=1, coef=', weighted_neighborhood_LIME(0, X, Y, 1))
print('d=10, coef=',weighted_neighborhood_LIME(0, X, Y, 10))
print('d=100, coef=',weighted_neighborhood_LIME(0, X, Y, 100))
plt.plot(X,Y)
plt.show()

# The Shapley Value

In [ ]:
X, y = shap.datasets.boston()
# Train a gradient-boosted decision tree on the underlying data 
X_train, X_test, y_train, y_test = train_test_split(X, y)
model = xgboost.XGBRegressor().fit(X_train, y_train)
print('R^2 of regression model: ', model.score(X_test, y_test))
# Generate Shapley values
## << YOUR CODE HERE >> 

In [ ]:
shap.plots.scatter(shap_values[:,"AGE"], color=shap_values)

Notice how as the age of the house increases, the contribution to the overall house price (measured by the SHAP value) starts to decrease, indicating that older houses are less valuable and the model uses age in a fairly monotonic way. 

# Attribution for Vision Models

### Load Model

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from trulens.nn.models import get_model_wrapper
keras_model = VGG16(weights='imagenet')
keras_model.summary()

In [ ]:
# Load the example record
url = 'https://upload.wikimedia.org/wikipedia/commons/e/e3/Mustela_nivalis_-British_Wildlife_Centre-4.jpg'
with PIL.Image.open(requests.get(url, stream=True).raw) as img:
    # View the image.
    plt.imshow(img)
    # Resize and transform the underlying array to pass it into the model 
    # x_pp = pass # << YOUR CODE HERE >> 
    x = np.array(img.resize((224,224), PIL.Image.ANTIALIAS))[np.newaxis]
    x_pp = preprocess_input(np.array(x, copy=True))
# Pretty-print the model's top 5 predictions on this record.
preds = keras_model.predict(x_pp)
print('Top classes: ', np.argsort(preds[0])[::-1][:5])
decode_predictions(preds)

### Saliency Maps

In [ ]:
from trulens.nn.attribution import InputAttribution
from trulens.nn.attribution import IntegratedGradients
from trulens.nn.attribution import InternalInfluence
from trulens.visualizations import MaskVisualizer

def visualize_saliency_map(model, x_pp, x, output_channel, pixel_thresh=0.9):
    # model: underlying keras model
    # x_pp: preprocessed image to be fed into model
    # x: unprocessed raw image (used to visualize saliency map on top of)
    # output_channel: integer representing class index of output channel to calculate explanations with respect to
    # pixel_thresh: float threshold to pass into MaskVisualizer for which pixels to display
    # << YOUR CODE HERE >> 
    pass 

### Integrated Gradients

In [ ]:
def visualize_integrated_gradients(model, x_pp, x, baseline=None, pixel_thresh=0.9):
    # model: underlying keras model
    # x_pp: preprocessed image to be fed into model
    # x: unprocessed raw image (used to visualize saliency map on top of)
    # baseline: np.array of same shape as x_pp, used as baseline in integrated gradients
    # pixel_thresh: float threshold to pass into MaskVisualizer for which pixels to display
    # << YOUR CODE HERE >> 
    pass 

In [ ]:
print('Integrated Gradients with black baseline: ')
# << YOUR CALL TO VISUALIZE_INTEGRATED_GRADIENTS HERE >> 

In [ ]:
print('Integrated Gradients with white baseline: ')
# << YOUR CALL TO VISUALIZE_INTEGRATED_GRADIENTS HERE >> 

In [ ]:
print('Integrated Gradients with random baseline: ')
# << YOUR CALL TO VISUALIZE_INTEGRATED_GRADIENTS HERE >> 

### Internal Influence

In [ ]:
from trulens.nn.distributions import LinearDoi, PointDoi

def get_internal_influences(model, x_pp, x, layer_name, doi, qoi):
    # model: underlying keras model
    # x_pp: preprocessed image to be fed into model
    # x: unprocessed raw image (used to visualize saliency map on top of)
    # layer_name: layer from which to calculations attributions with respect to
    # doi: Distribution of Interest for internal influence calculations; see trulens docs 
    # qoi: Quantiy of Interest for internal influence calculations; see trulens docs 
    # << YOUR CODE HERE >> 
    pass 

In [ ]:
# << YOUR CODE HERE >> 
# Compare internal influences for the 'weasel' class (most probable) using two DoIs

In [ ]:
# << YOUR CODE HERE >> 
# Compare internal influences for the 'polecat' class (second most probable) using two DoIs

In [ ]:
# << YOUR CODE HERE >> 
# Calculate the comparative internal influences between the weasel and polecat classes using two DoIs

### Comparing Attribution Methods

#### Visual Comparisons

In [ ]:
!wget https://hw2dataset.s3.amazonaws.com/ImageNet_1K.npy
imagenet_sample = np.load('ImageNet_1K.npy')
# NOTE: These imagenet samples have ALREADY been preprocessed (do not call preprocess_input)

#### Average Drop Percentage

In [ ]:
flamingo_class_index = 130
flamingo_image_indices = [210, 238, 240, 344, 354, 506, 907]

In [ ]:
from keras.models import Model, Sequential
from keras.layers import Dense, Input

def get_attribution_map_saliency(input_image, class_index):
    # << YOUR CODE HERE >> 
    pass 

def get_attribution_map_integrated_gradients(input_image, class_index):
    ## << YOUR CODE HERE >>
    pass 

def get_pre_softmax_output_score(input_image, class_index):
    # Hint: There are many ways to do this, but one way is to create a new model
    # that gets the input into the predictions layer and then applies the predictions layer without the softmax
    # << YOUR CODE HERE >> 
    pass 

def calculate_drop_single_image(input_image, class_index, attribution_type='saliency'):
    ## << YOUR CODE HERE >> 
    ## Attribution type is one of 'saliency' or 'integrated_gradients'
    ## Remember to copy the input image if modifying it
    pass 

def calculate_average_drop(input_images, class_index, attribution_type):
    # input images: array of preprocessed images 
    # class index to calculate attributions with respect to (integer)
    # attribution type: one of 'saliency' or 'integrated_gradients'
    pass 

In [ ]:
print('Average Drop % [SALIENCY MAP]:', calculate_average_drop(imagenet_sample[flamingo_image_indices], flamingo_class_index, 'saliency'))
print('Average Drop % [INTEGRATED GRADIENTS]:', calculate_average_drop(imagenet_sample[flamingo_image_indices], flamingo_class_index, 'integrated_gradients'))